In [5]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma



In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
import pandas as pd
books = pd.read_csv('books_cleaned.csv')

In [2]:
books["tagged_description"].to_csv('tagged_description.txt', sep='\n', index=False, header=False)

In [7]:
raw_documents = TextLoader('tagged_description.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1170, which is longer than the specified 0
Created a chunk of size 1216, which is longer than the specified 0
Created a chunk of size 375, which is longer than the specified 0
Created a chunk of size 311, which is longer than the specified 0
Created a chunk of size 485, which is longer than the specified 0
Created a chunk of size 484, which is longer than the specified 0
Created a chunk of size 962, which is longer than the specified 0
Created a chunk of size 190, which is longer than the specified 0
Created a chunk of size 845, which is longer than the specified 0
Created a chunk of size 298, which is longer than the specified 0
Created a chunk of size 199, which is longer than the specified 0
Created a chunk of size 883, which is longer than the specified 0
Created a chunk of size 1090, which is longer than the specified 0
Created a chunk of size 1191, which is longer than the specified 0
Created a chunk of size 306, which is longer than the specified 0
Create

In [8]:
db_books= Chroma.from_documents(documents, OpenAIEmbeddings())

/tmp/ipykernel_75697/2637766849.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db_books= Chroma.from_documents(documents, OpenAIEmbeddings())


ValidationError: 1 validation error for OpenAIEmbeddings
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'cli...20, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)
docs

In [ ]:
books[books["isbn13"]] == int(docs[0].page_content.split()[0].strip())  # Extract ISBN13 from the first document

In [ ]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
)--> pd.DataFrame:
    recs = db_books.similarity_search(query, k= 50)

    books_list = []

    for i in range(0,len(recs)):
        books_list += [int(recs[i].page_content.strip('"')split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)

